In [22]:
from tkinter import Tk
from tkinter import *
import random

root = Tk()
root.title("Glow_hockey")
root.geometry('1370x750')

''

In [23]:

AI_control = 0
pl_control = 0

width = 1370
height = 750
barrier = 25
goal_size = 350

indent = 100
st_vel = 20
goal_up = height // 2 - goal_size // 2
goal_down = height // 2 + goal_size // 2
pad_size = 50
puck_size = 30
AI_score = 0
pl_score = 0

AI_x_start = barrier + indent - pad_size // 2
AI_y_start = height // 2 - pad_size // 2

pl_x_start = width - (barrier + indent + pad_size // 2)
pl_y_start = height // 2 - pad_size // 2

puck_x_start = width // 2 - puck_size  // 2 + 30
puck_y_start = height // 2 - puck_size // 2

In [24]:
canv = Canvas(bg = 'red')

canv.create_rectangle(barrier, barrier , width - barrier, height - barrier, fill = 'black')
canv.create_rectangle(0, goal_up, barrier, goal_down, fill = 'cyan')
canv.create_rectangle(width - barrier, goal_up, width, goal_down, fill = 'cyan')
canv.create_line(width // 2, barrier, width // 2, height - barrier, fill = 'orange')
canv.create_oval(AI_x_start , AI_y_start , AI_x_start + pad_size , AI_y_start + pad_size , fill = 'blue', tag = 'AI')
canv.create_oval(pl_x_start , pl_y_start , pl_x_start + pad_size , pl_y_start + pad_size , fill = 'magenta', tag = 'pl')
canv.create_oval(puck_x_start, puck_y_start , puck_x_start + puck_size , puck_y_start + puck_size , fill = 'pink', tag = 'puck')
canv.create_text(100, 100, text = AI_score, font="Arial 20", fill = "white", tag = "AI_score")
canv.create_text(800, 100, text = pl_score, font="Arial 20", fill = "white", tag = "pl_score")

canv.pack(fill = BOTH, expand = 1)

AI_x = AI_x_start; AI_y = AI_y_start
pl_x = pl_x_start; pl_y = pl_y_start
puck_x = puck_x_start; puck_y = puck_y_start

vel_AI_x = 0; vel_AI_y = 0
vel_pl_x = 0; vel_pl_y = 0
vel_puck_x = 0; vel_puck_y = 0

In [25]:
def AI():
    global pl_control, puck_x, puck_y, vel_puck_x, vel_puck_y
    global AI_x, AI_y, vel_AI_x, vel_AI_y, AI_control
    
    if vel_puck_x == 0 and vel_puck_y == 0 and puck_x < width // 2 and not AI_control:
        vel_AI_x = st_vel
        vel_AI_y = 0
    elif AI_control:
        AI_control = 0
        vel_puck_x = 2 * st_vel
        vel_puck_y = 2 * st_vel * (random.randint(1, 3) - 2)
        puck_x += vel_puck_x * 2
        puck_y += vel_puck_y * 2
    elif vel_puck_x == 0 or pl_control:
        if abs((height // 2 - AI_y - pad_size // 2) * 2 - (height // 2 - puck_y - puck_size // 2)) >= 2 * st_vel:
            if (height // 2 - AI_y - pad_size // 2) * 2 > (height // 2 - puck_y - puck_size // 2):
                vel_AI_y = st_vel
            else:
                vel_AI_y = - st_vel
        else:
            vel_AI_y = 0
        
        if abs ((AI_x + pad_size // 2 - barrier) * 2 - (width - barrier - (puck_x + puck_size // 2))) >= 2 * st_vel:
            if (AI_x + pad_size // 2 - barrier) * 2 > (width - barrier - (puck_x + puck_size // 2)):
                vel_AI_x = - st_vel
            else:
                vel_AI_x = st_vel
        else:
            vel_AI_x = 0
        
    elif not pl_control and vel_puck_x != 0:
        tmp_x, tmp_y, vel_tmp_x, vel_tmp_y = puck_x, puck_y, vel_puck_x, vel_puck_y
        for t in range (100):
            tmp_x += vel_tmp_x
            tmp_y += vel_tmp_y
            
            if tmp_y < barrier:
                tmp_y = barrier
                vel_tmp_y = - vel_tmp_y
            if tmp_y > height - barrier - puck_size:
                tmp_y = height - barrier - puck_size
                vel_tmp_y = - vel_tmp_y
            
            if tmp_x < barrier:
                tmp_x = barrier
                vel_tmp_x = - vel_tmp_x
                
            if tmp_x > width - barrier - puck_size:
                tmp_x = width - barrier - puck_size
                vel_tmp_x = - vel_tmp_x
                
            
            if max(abs(tmp_x - AI_x), abs(tmp_y - AI_y)) < t * st_vel + pad_size // 2 + puck_size // 2 and tmp_x < width // 2:
                if AI_x < tmp_x - st_vel // 2:
                    vel_AI_x = st_vel
                else:
                    vel_AI_x= - st_vel
                    
                if AI_y < tmp_y - st_vel // 2:
                    vel_AI_y = st_vel
                else:
                    vel_AI_y = - st_vel
                break

In [26]:
def update_coords():
    global AI_x, AI_y, vel_AI_x, vel_AI_y
    global pl_x, pl_y, vel_pl_x, vel_pl_y
    global puck_x, puck_y, vel_puck_x, vel_puck_y
    global AI_control, pl_control
    
    center_AI_x = AI_x + pad_size // 2
    center_AI_y = AI_y + pad_size // 2
    center_pl_x = pl_x + pad_size // 2
    center_pl_y = pl_y + pad_size // 2
    center_puck_x = puck_x + puck_size // 2
    center_puck_y = puck_y + puck_size // 2
    
    if (center_AI_x - center_puck_x)**2 + (center_AI_y - center_puck_y)**2 <= ((pad_size + puck_size) // 2)**2:
        AI_control = 1
        
    if (center_pl_x - center_puck_x)**2 + (center_pl_y - center_puck_y)**2 <= ((pad_size + puck_size) // 2)**2:
        pl_control = 1
    
    if AI_control:
        puck_x = AI_x + pad_size // 2 - puck_size // 2
        puck_y = AI_y + pad_size // 2 - puck_size // 2
        vel_puck_x = vel_AI_x
        vel_puck_y = vel_AI_y
    if pl_control:
        puck_x = pl_x + pad_size // 2 - puck_size // 2
        puck_y = pl_y + pad_size // 2 - puck_size // 2
        vel_puck_x = vel_pl_x
        vel_puck_y = vel_pl_y
        
    AI_x += vel_AI_x; AI_y += vel_AI_y
    pl_x += vel_pl_x; pl_y += vel_pl_y
    puck_x += vel_puck_x; puck_y += vel_puck_y

In [27]:
def collision():
    global AI_x, AI_y, vel_AI_x, vel_AI_y
    global pl_x, pl_y, vel_pl_x, vel_pl_y
    global puck_x, puck_y, vel_puck_x, vel_puck_y
    global AI_control, pl_control
    global AI_score, pl_score
    
    if AI_x < barrier:
        AI_x = barrier
    if AI_y < barrier:
        AI_y = barrier
    if AI_x + pad_size > width // 2:
        AI_x = width // 2 - pad_size
    if AI_y + pad_size > height - barrier:
        AI_y = height - barrier - pad_size
    
    if pl_x < width // 2:
        pl_x = width // 2
        vel_pl_x = 0
    if pl_y < barrier:
        pl_y = barrier
        vel_pl_y = 0
    if pl_x + pad_size > width - barrier:
        pl_x = width - barrier - pad_size
        vel_pl_x = 0
    if pl_y + pad_size > height - barrier:
        pl_y = height - barrier - pad_size
        vel_pl_y = 0
    
    if puck_x < barrier and goal_up < puck_y < goal_down - puck_size and not AI_control:
        puck_x = width // 2 - puck_size // 2 - 50
        puck_y = height // 2 - puck_size // 2
        AI_x = AI_x_start; AI_y = AI_y_start
        pl_x = pl_x_start; pl_y = pl_y_start
        vel_puck_x = - st_vel; vel_puck_y = 0
        AI_score += 1
    elif puck_x < barrier:
        puck_x = barrier
        vel_puck_x = - vel_puck_x
        
    if puck_x > width - barrier - puck_size and goal_up < puck_y < goal_down - puck_size and not pl_control:
        puck_x = width // 2 - puck_size // 2 + 50
        puck_y = height // 2 - puck_size // 2
        AI_x = AI_x_start; AI_y = AI_y_start
        pl_x = pl_x_start; pl_y = pl_y_start
        vel_puck_x = st_vel; vel_puck_y = 0
        pl_score += 1
        
    elif puck_x > width - barrier - puck_size:
        puck_x = width - barrier - puck_size
        vel_puck_x = - vel_puck_x
    
    if puck_y < barrier:
        puck_y = barrier
        vel_puck_y = - vel_puck_y
    if puck_y > height - barrier - puck_size:
        puck_y = height - barrier - puck_size
        vel_puck_y = - vel_puck_y

In [28]:
def repaint():
    canv.delete('AI')
    canv.delete('pl')
    canv.delete('puck')
    canv.delete('AI_score')
    canv.delete('pl_score')
    
    canv.create_oval(AI_x, AI_y, AI_x + pad_size, AI_y + pad_size, fill = 'blue', tag = 'AI')
    canv.create_oval(pl_x, pl_y, pl_x + pad_size, pl_y + pad_size, fill = 'magenta', tag = 'pl')
    canv.create_oval(puck_x, puck_y, puck_x + puck_size, puck_y + puck_size, fill = 'pink', tag = 'puck')
    canv.create_text(100, 100, text = pl_score, font="Arial 20", fill = "white", tag = 'AI_score')
    canv.create_text(800, 100, text = AI_score, font="Arial 20", fill = "white", tag = 'pl_score')

In [29]:
def update_env():    
    AI()
    
    update_coords()
    
    collision()
    
    repaint()

def main():
    update_env()
    root.after(30, main)

canv.focus_set()

In [30]:
def movement_handler(event):
    global vel_pl_x, vel_pl_y, pl_control, puck_x, puck_y, vel_puck_x, vel_puck_y
    
    if event.keysym == "Up":
        vel_pl_y = - st_vel
    if event.keysym == "Down":
        vel_pl_y = st_vel
    if event.keysym == "Left":
        vel_pl_x = - st_vel
    if event.keysym == "Right":
        vel_pl_x = st_vel
    if event.keysym == "Return" and pl_control:
        pl_control = 0
        vel_puck_x = 2 * vel_pl_x
        vel_puck_y = 2 * vel_pl_y
        puck_x += vel_puck_x * 2
        puck_y += vel_puck_y * 2
        
canv.bind("<KeyPress>", movement_handler)

'2859544671296movement_handler'

In [31]:
def stop_pad(event):
    global vel_pl_x, vel_pl_y
    
    if event.keysym in ("Left", "Right"):
        vel_pl_x = 0
    if event.keysym in ("Up", "Down"):
        vel_pl_y = 0
    
    
canv.bind("<KeyRelease>", stop_pad)

main()
 
mainloop()